In [1]:
import random
import pandas as pd
import numpy as np
import os

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [36]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [37]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].apply(lambda x : int(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : int(x[9:11]))

In [38]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['date'] = train_df['일시'].apply(lambda x : int(x[0:8]))

In [11]:
train_df[train_df['건물번호'] == 2].head(24)

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),month,day,time,date
2040,2_20240601 00,2,20240601 00,18.3,0.0,2.6,82.0,0.0,0.00,1204.26,6,1,0,20240601
2041,2_20240601 01,2,20240601 01,18.3,0.0,2.7,82.0,0.0,0.00,1049.82,6,1,1,20240601
2042,2_20240601 02,2,20240601 02,18.1,0.0,2.6,80.0,0.0,0.00,1005.48,6,1,2,20240601
2043,2_20240601 03,2,20240601 03,18.0,0.0,2.6,81.0,0.0,0.00,1017.30,6,1,3,20240601
2044,2_20240601 04,2,20240601 04,17.8,0.0,1.3,81.0,0.0,0.00,969.24,6,1,4,20240601
2045,2_20240601 05,2,20240601 05,17.7,0.0,2.2,83.0,0.0,0.00,971.82,6,1,5,20240601
2046,2_20240601 06,2,20240601 06,17.6,0.0,1.7,85.0,0.0,0.02,936.18,6,1,6,20240601
2047,2_20240601 07,2,20240601 07,17.7,0.0,1.2,81.0,0.0,0.22,896.82,6,1,7,20240601
2048,2_20240601 08,2,20240601 08,18.2,0.0,1.8,78.0,0.0,0.56,920.16,6,1,8,20240601
2049,2_20240601 09,2,20240601 09,19.4,0.0,2.4,70.0,0.4,1.27,972.24,6,1,9,20240601


In [7]:
train_df[train_df['일시'] == '20240601 18']

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),month,day,time
18,1_20240601 18,1,20240601 18,23.8,0.0,2.6,45.0,1.0,1.44,5985.18,6,1,18
2058,2_20240601 18,2,20240601 18,23.8,0.0,2.6,45.0,1.0,1.44,973.02,6,1,18
4098,3_20240601 18,3,20240601 18,23.8,0.0,2.6,45.0,1.0,1.44,11479.62,6,1,18
6138,4_20240601 18,4,20240601 18,23.8,0.0,2.6,45.0,1.0,1.44,822.84,6,1,18
8178,5_20240601 18,5,20240601 18,23.8,0.0,2.6,45.0,1.0,1.44,3670.08,6,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193818,96_20240601 18,96,20240601 18,23.8,0.0,2.6,45.0,1.0,1.44,750.24,6,1,18
195858,97_20240601 18,97,20240601 18,23.7,0.0,3.6,45.0,1.0,1.37,1236.96,6,1,18
197898,98_20240601 18,98,20240601 18,22.3,0.0,2.6,48.0,1.0,0.00,1850.40,6,1,18
199938,99_20240601 18,99,20240601 18,25.7,0.0,2.1,39.0,0.4,0.70,934.56,6,1,18


In [74]:
train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_y = train_df['전력소비량(kWh)']

In [ ]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204000 entries, 0 to 203999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   건물번호     204000 non-null  int64  
 1   기온(°C)   204000 non-null  float64
 2   강수량(mm)  204000 non-null  float64
 3   풍속(m/s)  204000 non-null  float64
 4   습도(%)    204000 non-null  float64
 5   month    204000 non-null  int64  
 6   day      204000 non-null  int64  
 7   time     204000 non-null  int64  
dtypes: float64(4), int64(4)
memory usage: 12.5 MB


In [ ]:
# 시계열 특성을 학습에 반영하기 위해 test 데이터도 동일하게 처리합니다
test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))

In [ ]:
test_x = test_df.drop(columns=['num_date_time', '일시'])

In [ ]:
model = XGBRegressor()
model.fit(train_x, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
preds = model.predict(test_x)

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = preds

In [ ]:
submission.to_csv('./baseline_submission.csv', index=False)

In [39]:
build_df = pd.read_csv('./building_info.csv')

In [40]:
build_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    object 
 5   ESS저장용량(kWh)  100 non-null    object 
 6   PCS용량(kW)     100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [41]:
build_df['건물유형'].value_counts()

,count
건물유형,
백화점,16
호텔,10
상용,10
학교,10
건물기타,10
병원,9
아파트,9
연구소,9
IDC(전화국),9


In [42]:
trainb_df = pd.merge(train_df, build_df, on='건물번호')

In [43]:
trainb_df

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),month,day,time,date,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20240601 00,1,20240601 00,18.3,0.0,2.6,82.0,0.0,0.00,5794.80,6,1,0,20240601,호텔,82912.71,77586.0,-,-,-
1,1_20240601 01,1,20240601 01,18.3,0.0,2.7,82.0,0.0,0.00,5591.85,6,1,1,20240601,호텔,82912.71,77586.0,-,-,-
2,1_20240601 02,1,20240601 02,18.1,0.0,2.6,80.0,0.0,0.00,5338.17,6,1,2,20240601,호텔,82912.71,77586.0,-,-,-
3,1_20240601 03,1,20240601 03,18.0,0.0,2.6,81.0,0.0,0.00,4554.42,6,1,3,20240601,호텔,82912.71,77586.0,-,-,-
4,1_20240601 04,1,20240601 04,17.8,0.0,1.3,81.0,0.0,0.00,3602.25,6,1,4,20240601,호텔,82912.71,77586.0,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20240824 19,100,20240824 19,29.1,0.0,4.4,76.0,0.4,0.18,3276.00,8,24,19,20240824,호텔,162070.24,152943.0,-,-,-
203996,100_20240824 20,100,20240824 20,28.6,0.0,3.7,74.0,0.0,0.00,3197.52,8,24,20,20240824,호텔,162070.24,152943.0,-,-,-
203997,100_20240824 21,100,20240824 21,28.3,0.0,2.9,74.0,0.0,0.00,3006.60,8,24,21,20240824,호텔,162070.24,152943.0,-,-,-
203998,100_20240824 22,100,20240824 22,28.0,0.0,1.7,76.0,0.0,0.00,2649.72,8,24,22,20240824,호텔,162070.24,152943.0,-,-,-


In [ ]:
trainb_df[(trainb_df['건물유형'] == '호텔') & (trainb_df['일시'] == '20240824 19')]

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),...,day,time,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),냉방면적비율,건물유형코드
2035,1_20240824 19,1,20240824 19,29.3,0.0,2.7,75.0,0.0,0.32,7216.53,...,24,19,호텔,82912.71,77586.00,-,-,-,93.575521,0
8155,4_20240824 19,4,20240824 19,29.3,0.0,2.7,75.0,0.0,0.32,1236.60,...,24,19,호텔,41813.29,23715.71,-,-,-,56.718115,0
18355,9_20240824 19,9,20240824 19,27.2,0.0,0.8,81.0,0.0,0.00,3349.57,...,24,19,호텔,136757.00,109024.00,-,-,-,79.720965,0
20395,10_20240824 19,10,20240824 19,27.2,0.0,0.8,81.0,0.0,0.00,7915.40,...,24,19,호텔,435993.50,341983.00,-,-,-,78.437637,0
57115,28_20240824 19,28,20240824 19,28.8,0.0,1.7,74.0,0.0,0.06,1151.10,...,24,19,호텔,43894.83,5619.70,-,-,-,12.802647,0
157075,77_20240824 19,77,20240824 19,28.6,0.0,1.2,78.0,0.4,0.00,1995.66,...,24,19,호텔,79953.78,58489.31,-,-,-,73.153902,0
171355,84_20240824 19,84,20240824 19,26.6,0.0,2.5,83.0,0.0,0.08,2378.88,...,24,19,호텔,95232.88,23603.00,-,-,-,24.784507,0
181555,89_20240824 19,89,20240824 19,29.4,0.0,2.0,75.0,0.4,0.23,8891.12,...,24,19,호텔,303737.00,217825.56,77.38,-,-,71.715188,0
199915,98_20240824 19,98,20240824 19,27.0,0.0,1.0,78.0,0.3,0.00,2588.04,...,24,19,호텔,99102.62,22097.00,-,-,-,22.297090,0
203995,100_20240824 19,100,20240824 19,29.1,0.0,4.4,76.0,0.4,0.18,3276.00,...,24,19,호텔,162070.24,152943.00,-,-,-,94.368343,0


In [55]:
trainb_df['냉방면적비율'] = (trainb_df['냉방면적(m2)'] / trainb_df['연면적(m2)']) * 100

In [56]:
# '색상' 컬럼을 숫자로 인코딩하고 새로운 컬럼 '색상_인코딩' 추가
trainb_df['건물유형코드'], unique_labels = pd.factorize(trainb_df['건물유형'])


In [82]:
trainb_df

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),...,time,date,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),냉방면적비율,건물유형코드
0,1_20240601 00,1,20240601 00,18.3,0.0,2.6,82.0,0.0,0.00,5794.80,...,0,20240601,hotel,82912.71,77586.0,0,0,0,93.575521,0
1,1_20240601 01,1,20240601 01,18.3,0.0,2.7,82.0,0.0,0.00,5591.85,...,1,20240601,hotel,82912.71,77586.0,0,0,0,93.575521,0
2,1_20240601 02,1,20240601 02,18.1,0.0,2.6,80.0,0.0,0.00,5338.17,...,2,20240601,hotel,82912.71,77586.0,0,0,0,93.575521,0
3,1_20240601 03,1,20240601 03,18.0,0.0,2.6,81.0,0.0,0.00,4554.42,...,3,20240601,hotel,82912.71,77586.0,0,0,0,93.575521,0
4,1_20240601 04,1,20240601 04,17.8,0.0,1.3,81.0,0.0,0.00,3602.25,...,4,20240601,hotel,82912.71,77586.0,0,0,0,93.575521,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20240824 19,100,20240824 19,29.1,0.0,4.4,76.0,0.4,0.18,3276.00,...,19,20240824,hotel,162070.24,152943.0,0,0,0,94.368343,0
203996,100_20240824 20,100,20240824 20,28.6,0.0,3.7,74.0,0.0,0.00,3197.52,...,20,20240824,hotel,162070.24,152943.0,0,0,0,94.368343,0
203997,100_20240824 21,100,20240824 21,28.3,0.0,2.9,74.0,0.0,0.00,3006.60,...,21,20240824,hotel,162070.24,152943.0,0,0,0,94.368343,0
203998,100_20240824 22,100,20240824 22,28.0,0.0,1.7,76.0,0.0,0.00,2649.72,...,22,20240824,hotel,162070.24,152943.0,0,0,0,94.368343,0


In [83]:
train_x = trainb_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '습도(%)'])

In [84]:
train_x

,건물번호,기온(°C),강수량(mm),풍속(m/s),month,day,time,date,냉방면적비율,건물유형코드
0,1,18.3,0.0,2.6,6,1,0,20240601,93.575521,0
1,1,18.3,0.0,2.7,6,1,1,20240601,93.575521,0
2,1,18.1,0.0,2.6,6,1,2,20240601,93.575521,0
3,1,18.0,0.0,2.6,6,1,3,20240601,93.575521,0
4,1,17.8,0.0,1.3,6,1,4,20240601,93.575521,0
...,...,...,...,...,...,...,...,...,...,...
203995,100,29.1,0.0,4.4,8,24,19,20240824,94.368343,0
203996,100,28.6,0.0,3.7,8,24,20,20240824,94.368343,0
203997,100,28.3,0.0,2.9,8,24,21,20240824,94.368343,0
203998,100,28.0,0.0,1.7,8,24,22,20240824,94.368343,0


In [62]:
test_df

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%)
0,1_20240825 00,1,20240825 00,26.5,0.0,0.7,80.0
1,1_20240825 01,1,20240825 01,26.1,0.0,0.0,80.0
2,1_20240825 02,1,20240825 02,25.9,0.0,0.3,83.0
3,1_20240825 03,1,20240825 03,25.7,0.0,1.1,83.0
4,1_20240825 04,1,20240825 04,25.5,0.0,1.0,86.0
...,...,...,...,...,...,...,...
16795,100_20240831 19,100,20240831 19,27.1,0.0,2.7,68.0
16796,100_20240831 20,100,20240831 20,26.5,0.0,2.9,73.0
16797,100_20240831 21,100,20240831 21,26.3,0.0,3.5,73.0
16798,100_20240831 22,100,20240831 22,26.0,0.0,3.4,68.0


In [93]:

#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))
test_df['date'] = test_df['일시'].apply(lambda x : int(x[0:8]))

testb_df = pd.merge(test_df, build_df, on='건물번호')

In [94]:
testb_df['냉방면적비율'] = (testb_df['냉방면적(m2)'] / testb_df['연면적(m2)']) * 100

In [95]:
# '색상' 컬럼을 숫자로 인코딩하고 새로운 컬럼 '색상_인코딩' 추가
testb_df['건물유형코드'], unique_labels = pd.factorize(testb_df['건물유형'])

In [96]:
testb_df

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),month,day,time,date,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),냉방면적비율,건물유형코드
0,1_20240825 00,1,20240825 00,26.5,0.0,0.7,80.0,8,25,0,20240825,호텔,82912.71,77586.0,-,-,-,93.575521,0
1,1_20240825 01,1,20240825 01,26.1,0.0,0.0,80.0,8,25,1,20240825,호텔,82912.71,77586.0,-,-,-,93.575521,0
2,1_20240825 02,1,20240825 02,25.9,0.0,0.3,83.0,8,25,2,20240825,호텔,82912.71,77586.0,-,-,-,93.575521,0
3,1_20240825 03,1,20240825 03,25.7,0.0,1.1,83.0,8,25,3,20240825,호텔,82912.71,77586.0,-,-,-,93.575521,0
4,1_20240825 04,1,20240825 04,25.5,0.0,1.0,86.0,8,25,4,20240825,호텔,82912.71,77586.0,-,-,-,93.575521,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100_20240831 19,100,20240831 19,27.1,0.0,2.7,68.0,8,31,19,20240831,호텔,162070.24,152943.0,-,-,-,94.368343,0
16796,100_20240831 20,100,20240831 20,26.5,0.0,2.9,73.0,8,31,20,20240831,호텔,162070.24,152943.0,-,-,-,94.368343,0
16797,100_20240831 21,100,20240831 21,26.3,0.0,3.5,73.0,8,31,21,20240831,호텔,162070.24,152943.0,-,-,-,94.368343,0
16798,100_20240831 22,100,20240831 22,26.0,0.0,3.4,68.0,8,31,22,20240831,호텔,162070.24,152943.0,-,-,-,94.368343,0


In [97]:


test_x = testb_df.drop(columns=['num_date_time', '일시', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '습도(%)'])

In [98]:
test_x

,건물번호,기온(°C),강수량(mm),풍속(m/s),month,day,time,date,냉방면적비율,건물유형코드
0,1,26.5,0.0,0.7,8,25,0,20240825,93.575521,0
1,1,26.1,0.0,0.0,8,25,1,20240825,93.575521,0
2,1,25.9,0.0,0.3,8,25,2,20240825,93.575521,0
3,1,25.7,0.0,1.1,8,25,3,20240825,93.575521,0
4,1,25.5,0.0,1.0,8,25,4,20240825,93.575521,0
...,...,...,...,...,...,...,...,...,...,...
16795,100,27.1,0.0,2.7,8,31,19,20240831,94.368343,0
16796,100,26.5,0.0,2.9,8,31,20,20240831,94.368343,0
16797,100,26.3,0.0,3.5,8,31,21,20240831,94.368343,0
16798,100,26.0,0.0,3.4,8,31,22,20240831,94.368343,0


In [85]:
model2 = XGBRegressor()
model2.fit(train_x, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [99]:
preds = model2.predict(test_x)

In [100]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = preds

In [101]:
submission.to_csv('./baseline_submission_20250811_01.csv', index=False)

In [46]:
trainb_df['건물유형'].unique()

array(['hotel', 'commerce', 'hospital', 'school', 'etc7', 'apartment',
       'lab', 'department', 'etc26', 'idc', 'public', 'etc47', 'etc58',
       'etc61', 'etc69', 'etc78', 'etc82', 'etc96', 'etc97'], dtype=object)

In [45]:
trainb_df.loc[trainb_df['건물유형']=='호텔', '건물유형'] = 'hotel'
trainb_df.loc[trainb_df['건물유형']=='상용', '건물유형'] = 'commerce'
trainb_df.loc[trainb_df['건물유형']=='병원', '건물유형'] = 'hospital'
trainb_df.loc[trainb_df['건물유형']=='학교', '건물유형'] = 'school'
trainb_df.loc[trainb_df['건물유형']=='아파트', '건물유형'] = 'apartment'
trainb_df.loc[trainb_df['건물유형']=='백화점', '건물유형'] = 'department'
trainb_df.loc[trainb_df['건물유형']=='IDC(전화국)', '건물유형'] = 'idc'
trainb_df.loc[trainb_df['건물유형']=='공공', '건물유형'] = 'public'
trainb_df.loc[trainb_df['건물유형']=='연구소', '건물유형'] = 'lab'
trainb_df.loc[trainb_df['건물유형']=='건물기타', '건물유형'] = 'etc' + trainb_df[trainb_df['건물유형']=='건물기타']['건물번호'].astype(str)

In [51]:
trainb_df[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']] = trainb_df[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']].replace('-', 0)

In [52]:
trainb_df

,num_date_time,건물번호,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),month,day,time,date,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20240601 00,1,20240601 00,18.3,0.0,2.6,82.0,0.0,0.00,5794.80,6,1,0,20240601,hotel,82912.71,77586.0,0,0,0
1,1_20240601 01,1,20240601 01,18.3,0.0,2.7,82.0,0.0,0.00,5591.85,6,1,1,20240601,hotel,82912.71,77586.0,0,0,0
2,1_20240601 02,1,20240601 02,18.1,0.0,2.6,80.0,0.0,0.00,5338.17,6,1,2,20240601,hotel,82912.71,77586.0,0,0,0
3,1_20240601 03,1,20240601 03,18.0,0.0,2.6,81.0,0.0,0.00,4554.42,6,1,3,20240601,hotel,82912.71,77586.0,0,0,0
4,1_20240601 04,1,20240601 04,17.8,0.0,1.3,81.0,0.0,0.00,3602.25,6,1,4,20240601,hotel,82912.71,77586.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20240824 19,100,20240824 19,29.1,0.0,4.4,76.0,0.4,0.18,3276.00,8,24,19,20240824,hotel,162070.24,152943.0,0,0,0
203996,100_20240824 20,100,20240824 20,28.6,0.0,3.7,74.0,0.0,0.00,3197.52,8,24,20,20240824,hotel,162070.24,152943.0,0,0,0
203997,100_20240824 21,100,20240824 21,28.3,0.0,2.9,74.0,0.0,0.00,3006.60,8,24,21,20240824,hotel,162070.24,152943.0,0,0,0
203998,100_20240824 22,100,20240824 22,28.0,0.0,1.7,76.0,0.0,0.00,2649.72,8,24,22,20240824,hotel,162070.24,152943.0,0,0,0


In [54]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 건물유형과 건물번호로 정렬하여 unique 건물번호 가져오기
buildings = trainb_df.sort_values(['건물유형', '건물번호'])['건물번호'].unique()

# 각 건물별로 그래프 그리기
for building in buildings:
    # 해당 건물 데이터 추출
    building_data = trainb_df[trainb_df['건물번호'] == building]

    # 그래프 생성
    fig, ax1 = plt.subplots(figsize=(12, 6))

    # 첫 번째 y축 (기온)
    color1 = 'red'
    ax1.set_xlabel('date')
    ax1.set_ylabel('temper(°C)', color=color1)

    # 각 날짜별로 기온 데이터 플로팅
    for date in building_data['date'].unique():
        daily_data = building_data[building_data['date'] == date]
        ax1.plot(daily_data['time'], daily_data['기온(°C)'], color=color1, alpha=0.3)
    ax1.tick_params(axis='y', labelcolor=color1)

    # 두 번째 y축 (습도)
    ax2 = ax1.twinx()
    color2 = 'blue'
    ax2.set_ylabel('humid', color=color2)

    # 각 날짜별로 습도 데이터 플로팅
    for date in building_data['date'].unique():
        daily_data = building_data[building_data['date'] == date]
        ax2.plot(daily_data['time'], daily_data['습도(%)'], color=color2, alpha=0.3)
    ax2.tick_params(axis='y', labelcolor=color2)

    # 세 번째 y축 (소비전력량)
    ax3 = ax1.twinx()
    color3 = 'green'
    # 오른쪽 여백을 확보하기 위해 스파인 위치 조정
    ax3.spines['right'].set_position(('outward', 60))
    ax3.set_ylabel('power', color=color3)

    # 각 날짜별로 소비전력량 데이터 플로팅
    for date in building_data['date'].unique():
        daily_data = building_data[building_data['date'] == date]
        ax3.plot(daily_data['time'], daily_data['전력소비량(kWh)'], color=color3, alpha=0.3)
    ax3.tick_params(axis='y', labelcolor=color3)

    # x축 설정
    plt.xlabel('time')
    ax1.set_xticks(range(24))

    # 제목 설정
    cell = trainb_df[trainb_df['건물번호'] == building]

    plt.title(f"{cell['건물유형'].iloc[0]}/ESS-{cell['PCS용량(kW)'].iloc[0]}/SUN-{cell['태양광용량(kW)'].iloc[0]} - date,temper,humid,power")

    # 범례 설정
    # 각 변수별 더미 라인을 생성하여 범례에 표시
    dummy_lines = [
        plt.Line2D([0], [0], color=color1, label='temper'),
        plt.Line2D([0], [0], color=color2, label='humid'),
        plt.Line2D([0], [0], color=color3, label='power')
    ]
    ax1.legend(handles=dummy_lines, loc='upper right')

    # 그리드 추가
    ax1.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [105]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np

def train_with_monitoring(train_x, train_y, params=None):
    if params is None:
        params = {
            'n_estimators': 3000,
            'learning_rate': 0.01,
            'max_depth': 5,
            'early_stopping_rounds': 50,
            'eval_metric': ['rmse', 'mae']
        }

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(
        train_x, train_y,
        test_size=0.2,
        random_state=42
    )

    # 모델 생성
    model = XGBRegressor(**params)

    # 학습
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        verbose=50  # 50회마다 출력
    )

    # 훈련 결과 평가
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)

    # 다양한 메트릭으로 평가
    metrics = {
        'train_rmse': np.sqrt(mean_squared_error(y_train, train_preds)),
        'val_rmse': np.sqrt(mean_squared_error(y_val, val_preds)),
        'train_mae': mean_absolute_error(y_train, train_preds),
        'val_mae': mean_absolute_error(y_val, val_preds)
    }

    print("\nFinal Metrics:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.4f}")

    print(f"\nBest iteration: {model.best_iteration}")

    return model, metrics


train_x = trainb_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '습도(%)'])
test_x = testb_df.drop(columns=['num_date_time', '일시', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '습도(%)'])



# 건물번호 리스트 생성
buildings = train_x['건물번호'].unique()

# submission 파일 로드
submission = pd.read_csv('./sample_submission.csv')

# 예측값을 저장할 딕셔너리 생성
predictions_dict = {}

# 건물별로 학습 및 예측
for building in buildings:
    print(f"Processing building {building}")

    # 학습 데이터 분리
    train_data = trainb_df[trainb_df['건물번호'] == building]
    test_data = testb_df[testb_df['건물번호'] == building]

    # 특성과 타겟 분리
    train_x = train_data.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '습도(%)', '건물번호'])
    train_y = train_data['전력소비량(kWh)']
    test_x = test_data.drop(columns=['num_date_time', '일시', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', '습도(%)', '건물번호'])

    # 모델 학습 및 모니터링
    model2, metrics = train_with_monitoring(
        train_x,
        train_y,
        params={
            'n_estimators': 3000,
            'learning_rate': 0.01,
            'max_depth': 5,
            'early_stopping_rounds': 50,
            'eval_metric': ['rmse', 'mae']
        }
    )

    # 예측
    preds = model2.predict(test_x)

    # 예측값을 인덱스와 함께 저장
    for idx, pred in zip(test_data.index, preds):
        predictions_dict[idx] = pred

# submission 파일에 예측값 할당 (인덱스 기준으로)
submission['answer'] = submission.index.map(predictions_dict)

# 결과 저장
submission.to_csv('baseline_submission_20250811_03.csv', index=False)

# 검증
print("Number of predictions:", len(predictions_dict))
print("Number of rows in submission:", len(submission))
print("Any missing values:", submission['answer'].isna().any())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[1200]	validation_0-rmse:266.71396	validation_0-mae:170.00801	validation_1-rmse:437.29207	validation_1-mae:272.34308
[1250]	validation_0-rmse:259.64363	validation_0-mae:165.28300	validation_1-rmse:432.50380	validation_1-mae:269.97378
[1300]	validation_0-rmse:253.07547	validation_0-mae:160.95540	validation_1-rmse:428.21776	validation_1-mae:268.05817
[1350]	validation_0-rmse:247.75135	validation_0-mae:157.53708	validation_1-rmse:425.06364	validation_1-mae:266.74555
[1400]	validation_0-rmse:242.72649	validation_0-mae:154.20031	validation_1-rmse:422.22709	validation_1-mae:265.19351
[1450]	validation_0-rmse:237.53138	validation_0-mae:150.83524	validation_1-rmse:419.22755	validation_1-mae:263.42018
[1500]	validation_0-rmse:232.62498	validation_0-mae:147.56459	validation_1-rmse:416.70327	validation_1-mae:261.82714
[1550]	validation_0-rmse:228.56456	validation_0-mae:144.79565	validation_1-rmse:414.77845	validation_1-mae:260.99045
[1600]	validation_0-rmse:224